In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import os
import time
import io
import pyathena as cz
from pyathena.pandas_cursor import PandasCursor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [74]:
# s3 = boto3.resource('s3')
athena_conection = "athena.us-east-1.amazonaws.com"
region_name = ('us-east-1')
db = "ml_platform_events"

In [75]:
athena_client   = boto3.client('athena', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

s3_resource   = boto3.resource('s3', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

In [78]:
def athena_query_to_dataframe(db, query):
    
    listOfStatus = ['SUCCEEDED', 'FAILED', 'CANCELLED']
    listOfInitialStatus = ['RUNNING', 'QUEUED']
    
    print('Starting Query Execution:')
       
    response = athena_client.start_query_execution(
        QueryString = query,
        QueryExecutionContext = {
            'Database': db
        },
        ResultConfiguration = {
            'OutputLocation': 's3://personal-us-east-1/query-result/',
        }
    )

    queryExecutionId = response['QueryExecutionId']

    status = athena_client.get_query_execution(QueryExecutionId = queryExecutionId)['QueryExecution']['Status']['State']

    while status in listOfInitialStatus:
        status = athena_client.get_query_execution(QueryExecutionId = queryExecutionId)['QueryExecution']['Status']['State']
        if status in listOfStatus:
            if status == 'SUCCEEDED':
                print('Query Succeeded!')
                paginator = athena_client.get_paginator('get_query_results')
                query_results = paginator.paginate(
                    QueryExecutionId = queryExecutionId,
                    PaginationConfig = {'PageSize': 1000}
                )
            elif status == 'FAILED':
                print('Query Failed!')
            elif status == 'CANCELLED':
                print('Query Cancelled!')
            break
    
    results = []
    rows = []
    
    print('Processing Response')
    
    for page in query_results:
        for row in page['ResultSet']['Rows']:
            rows.append(row['Data'])

    columns = rows[0]
    rows = rows[1:]

    columns_list = []
    for column in columns:
        columns_list.append(column['VarCharValue'])
        
    print('Creating Dataframe')

    dataframe = pd.DataFrame(columns = columns_list)

    for row in rows:
        df_row = []
        for data in row:
            df_row.append(data['VarCharValue'])
        dataframe.loc[len(dataframe)] = df_row
    
        
    print('Done!')
    
    return(dataframe)

In [104]:
df = athena_query_to_dataframe(db, "select * from ml_platform_events.event_punkapi")

Starting Query Execution:
Query Succeeded!
Processing Response
Creating Dataframe
Done!


In [106]:
df = df.drop(['index', 'id'], axis=1)

In [107]:
df.head()

,name,abv,ibu,target_fg,target_og,ebc,srm,ph
0,Galaxy,6.7,67,1064,1013,19,9.5,4.4
1,10 Heads High,7.8,70,1015,1074,90,45,4.4
2,Lumberjack Stout,15.2,20,1020,1110,158,79,4.4
3,Neon Overlord,7.6,70,1012,1067,20,10,4.4
4,Ship Wreck,13.8,60,1015,1088,25,13,4.4


In [109]:
X = df.drop('ibu', axis=1)
y = df['ibu']

In [ ]:
train_test_split(X,y,test_size=0.33,random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)